<a href="https://colab.research.google.com/github/sallyjun08/WIDYOU/blob/master/League_of_legend_match_data_analyst_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'league-of-legendslol-ranked-games-2020-ver1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F586003%2F1093265%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240528%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240528T021425Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D558f403dc2731f8ee9ed029acf10c5caa2404252dfc64aaaffcd79b0e3bd3d7e079f324f0a8a48d1ce7f2a02e39ff2d4d8784d4405a53e747d4f56d76b0eda8db88d3a009debebd47fb85db17aa2850c76100bbf4e6e4b1cc589d4b0f4b2792049762c21553ea20a9dce2fae05ed955f1dc392768549d4cfa0f8b5abe29fe4484473dca5b2964413a96274158edc8169cd2d5c95e9de3840fda785ef4ac4ead1c7275b1dddfc448a20af0da039994d3dc73aa55c24b3bd3be561a5643ca831e9254e15065263483933fd386d8af13909e347d7dd26151da142d06fa938b86b90e374b5a22fcc7dd4fa64f59fe2abc9c375da3f885487df81f7b28f09e3708d8d'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import json
import re
import time
from pandas.io.json import json_normalize
from tqdm import tqdm_notebook

In [ ]:
match = pd.read_pickle("/kaggle/input/league-of-legendslol-ranked-games-2020-ver1/match_data_version1.pickle")

In [ ]:
winner_df =  pd.read_pickle('../input/league-of-legendslol-ranked-games-2020-ver1/match_winner_data_version1.pickle')
loser_df = pd.read_pickle('../input/league-of-legendslol-ranked-games-2020-ver1/match_loser_data_version1.pickle')

In [ ]:
match

In [ ]:
match = match[match["gameMode"]=="CLASSIC"]

In [ ]:
match

In [ ]:
match = match[["gameDuration","gameId","gameVersion","participants"]]

In [ ]:
match

In [ ]:
winner_df

In [ ]:
loser_df

In [ ]:
match.info()

In [ ]:
match

In [ ]:
match.columns

In [ ]:
match["gameType"].value_counts()

In [ ]:
match['participants'].iloc[0][0]

In [ ]:
match['participants'].iloc[0][0]["stats"]

In [ ]:
match = pd.merge(match,winner_df,on='gameId',how='left')

In [ ]:
match = pd.merge(match,loser_df,on='gameId',how='left')

In [ ]:
del [[winner_df, loser_df]]

In [ ]:
match = match[match["gameMode"]=="CLASSIC"]

In [ ]:
match.columns

In [ ]:
match.drop(["gameCreation","gameMode","gameType","mapId","platformId","status.message","status.status_code"],axis=1,inplace=True)

In [ ]:
match['gameVersion'] = match['gameVersion'].str.split(".").map(lambda x: "".join(x))

In [ ]:
match

In [ ]:
match.isnull().sum()

In [ ]:
match.to_pickle("matchpre.pkl")

In [ ]:
match = match.astype({"gameVersion":int})

In [ ]:
match["participantIdentities"].iloc[0]

In [ ]:
match["participants"].iloc[0]

In [ ]:
use_cols = ['kills', 'assists','totalDamageDealtToChampions','goldEarned', 'visionScore','timeCCingOthers']
stats_df = pd.DataFrame()
for i in tqdm_notebook(30000),desc="total"):
    temp = pd.DataFrame()
    for col in tqdm_notebook(use_cols,desc="columns"):
        cur_values = {f'{col}{j+1}': match['participants'].iloc[i][j]['stats'][col] for j in range(10)}
        temp = pd.concat([temp, pd.Series(cur_values)], axis=0, sort=False)
    stats_df = pd.concat([stats_df, temp], axis=1, sort=False)
stats_df = stats_df.T.reset_index(drop=True)
stats_df['gameId'] = match_df['gameId']


In [ ]:
stats_df

In [ ]:
stats_df.to_csv("stats.csv",index=False)

In [ ]:
vesion_date = pd.read_csv("/kaggle/input/lol-classic-rank-game-datakrtop-3-tier/lol_version_Date.csv")

match = pd.merge(match,vesion_date,how="inner",on="gameVersion")

match["date"] = pd.to_datetime(match["date"])

match["year"] = match["date"].dt.year
match["month"] = match["date"].dt.month
match["day"] = match["date"].dt.day